# Quick Guide

If you have not yet installed {{ Triumvirate }}, please follow the instructions
in ['Installation'](../installation.rst).

## Using the Python package

### Preamble

Load needed classes and functions from the relevant package modules:

In [1]:
from triumvirate.catalogue import ParticleCatalogue
from triumvirate.logger import setup_logger
from triumvirate.parameters import ParameterSet
from triumvirate.threept import compute_bispec_in_gpp_box

### Optional logger

A logger is useful for program tracking, and can be set up easily and passed
to callables that accept a ``logger`` argument:

In [2]:
# Set up the customised Triumvirate program logger.
trv_logger = setup_logger()

```{seealso}
[Customised Logger](./Logger.ipynb) for more details.

### Parameter configuration

The simplest way to initialise an instance of the parameter set class
{py:class}`~triumvirate.parameters.ParameterSet` is by loading an external
``.yml`` file. A template parameter file
[``"params_template.yml"``](../_static/params_template.yml) can be
downloaded, but the relevant parameters need to be changed accordingly
for your use.

```{seealso}
[Parameter Configuration](./Configuration.ipynb) and
[Parameter Set](./Parameters.ipynb) for more details.
```

In [3]:
import yaml

from triumvirate.parameters import fetch_paramset_template

params_dict = fetch_paramset_template('dict')

for ax_name in ['x', 'y', 'z']:
    params_dict['boxsize'][ax_name] = 1000.
    params_dict['ngrid'][ax_name] = 64

params_dict.update({
    'assignment'    : 'cic',
    'catalogue_type': 'survey',
    'statistic_type': 'bispec',
    'degrees'       : {'ell1': 0, 'ell2': 0, 'ELL': 0},
    'range'         : [0.005, 0.105],
    'num_bins'      : 10,
})

with open("params_template.yml", 'w') as params_file:
    yaml.dump(params_dict, params_file, default_flow_style=False)

Assuming the parameter file is in the current working directory, read the
parameter set from file:

In [4]:
parameter_set = ParameterSet("params_template.yml", logger=trv_logger)

[2024-08-19 23:53:17 (+00:00:00) INFO] Validating parameters... (entering C++)
[2024-08-19 23:53:17 (+00:00:00) INFO] ... validated parameters. (exited C++)
[2024-08-19 23:53:17 (+00:00:00) STAT C++] Parameters validated.


The parameters used here are (possibly printed in a different order):

In [5]:
parameter_set.print()

{'alignment': 'centre',
 'assignment': 'cic',
 'binning': 'lin',
 'boxsize': {'x': 1000.0, 'y': 1000.0, 'z': 1000.0},
 'catalogue_columns': [],
 'catalogue_type': 'survey',
 'cutoff_nyq': None,
 'degrees': {'ELL': 0, 'ell1': 0, 'ell2': 0},
 'directories': {'catalogues': '', 'measurements': ''},
 'expand': 1.0,
 'fftw_scheme': 'measure',
 'files': {'data_catalogue': '', 'rand_catalogue': ''},
 'form': 'diag',
 'idx_bin': None,
 'interlace': False,
 'ngrid': {'x': 64, 'y': 64, 'z': 64},
 'norm_convention': 'particle',
 'num_bins': 10,
 'padfactor': None,
 'padscale': 'box',
 'progbar': False,
 'range': [0.005, 0.105],
 'save_binned_vectors': False,
 'statistic_type': 'bispec',
 'tags': {'output': ''},
 'use_fftw_wisdom': False,
 'verbose': 20,
 'wa_orders': {'i': None, 'j': None},
 'volume': np.float64(1000000000.0),
 'nmesh': np.int64(262144),
 'assignment_order': 2,
 'npoint': '3pt',
 'space': 'fourier',
 'fftw_planner_flag': 0,
 'fftw_wisdom_file_f': '',
 'fftw_wisdom_file_b': ''}


### Catalogue data

We will measure clustering statistics from a single simulation-like catalogue
file ``"mock_catalogue_sim.dat"``, which contains three data columns ``'x'``,
``'y'`` and ``'z'`` for the Cartesian coordinates of particles.

In [6]:
import numpy as np

# Create simulation-like catalogue, or load if existing.
catalogue_sim_filepath = "mock_catalogue_sim.dat"
try:
    catalogue_sim = np.loadtxt(
        catalogue_sim_filepath,
        dtype=[(axis, np.float64) for axis in ['x', 'y', 'z']]
    )
except FileNotFoundError:
    from nbodykit.cosmology import Cosmology, LinearPower
    from nbodykit.lab import LogNormalCatalog

    # Cosmology, matter power spectrum and bias at given redshift
    cosmo = Cosmology(
        h=0.6736, Omega0_b=0.04930, Omega0_cdm=0.2645,
        A_s=2.083e-09, n_s=0.9649
    )
    redshift = 1.
    bias = 2.

    # Catalogue properties
    density = 5.e-4
    boxsize = 1000.

    # Catalogue selectors
    def cut_to_sphere(coords, boxsize):
        return np.less_equal(np.sqrt(np.sum(coords**2, axis=-1)), boxsize/2.)

    powspec = LinearPower(cosmo, redshift)
    catalogue_sim = LogNormalCatalog(
        powspec, density, boxsize, bias=bias, Nmesh=256, seed=42
    )

    catalogue_sim['Position'] -= boxsize/2.

    np.savetxt(catalogue_sim_filepath, catalogue_sim['Position'].compute())

Read the catalogue data from file:

In [7]:
# DEMO: warning messages are captured and reprinted here for demonstration.

import warnings  # DEMO

with warnings.catch_warnings(record=True) as demo_warnings:
    catalogue = ParticleCatalogue.read_from_file(
        "mock_catalogue_sim.dat",
        names=['x', 'y', 'z'],
        logger=trv_logger
    )
    for demo_warning in demo_warnings:
        print(demo_warning.message)

[2024-08-19 23:53:17 (+00:00:00) INFO] Original extents of particle coordinates: {'x': (-499.998, 499.998 | 999.996), 'y': (-500.000, 499.998 | 999.998), 'z': (-499.999, 500.000 | 999.999)} (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
[2024-08-19 23:53:17 (+00:00:00) INFO] Catalogue loaded: ntotal = 499214, wtotal = 499214.000, wstotal = 499214.000 (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
Catalogue 'nz' field is not provided and thus set to zero, which may raise errors in some computations.
Catalogue 'ws' field is not provided, so is set to unity.
Catalogue 'wc' field is not provided, so is set to unity.


```{hint}
As seen in the warning messages, catalogues should usually contain more
data columns than just the Cartesian coordinates. For a cuboid-box catalogue
of unweighted particles like the above, the Cartesian coordinates are the only
necessary input data.
```

```{seealso}
[Particle Catalogue](./Catalogue.ipynb) for more details.
```

### Clustering measurements

As an example, we measure the diagonal global plane-parallel bispectrum
monopole $(\ell_1, \ell_2, L) = (0, 0, 0)$ (set in the parameter set above)
from the simulation-like cuboid-box catalogue loaded above, and save the
result as a ``.txt`` file to the default output file ``"bk000_diag.txt"``.

Call the function:

In [8]:
results = compute_bispec_in_gpp_box(
    catalogue, paramset=parameter_set,
    save='.txt', logger=trv_logger
)

[2024-08-19 23:53:17 (+00:00:00) INFO] Validating parameters... (entering C++)
[2024-08-19 23:53:17 (+00:00:00) INFO] ... validated parameters. (exited C++)
[2024-08-19 23:53:17 (+00:00:00) STAT C++] Parameters validated.
[2024-08-19 23:53:17 (+00:00:00) INFO] Parameter set have been initialised.
[2024-08-19 23:53:17 (+00:00:00) INFO] Binning has been initialised.
[2024-08-19 23:53:17 (+00:00:00) INFO] Offset extents of particle coordinates: {'x': (0.002, 999.998 | 999.996), 'y': (0.001, 999.999 | 999.998), 'z': (0.000, 1000.000 | 999.999)} (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
[2024-08-19 23:53:17 (+00:00:00) INFO] Catalogue box has been periodised.
[2024-08-19 23:53:17 (+00:00:00) INFO] Inserted missing 'nz' field based on particle count and box size.
[2024-08-19 23:53:17 (+00:00:00) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2024-08-19 23:53:18 (+00:00:00) INFO] ... prepared catalogue for clustering algorithm. (exited C++)
[2024-08-19

The returned `results` is a dictionary containing the coordinate bin centres
and effective/average coordinate in each bin, the number of contributing
modes/pairs per bin and (the real and imaginary parts of) the raw measurements
including shot noise and the shot noise component:

In [9]:
# DEMO
from pprint import pprint
pprint(results)

{'bk_raw': array([-9.76634349e+08+2.00905377e-07j,  5.62308421e+09-4.85102525e-07j,
        8.69371048e+08-2.21547474e-07j,  1.28499830e+09-6.41153933e-08j,
        6.68698399e+08+7.29910031e-09j,  6.05966770e+08-7.01569721e-09j,
        3.85704714e+08-1.84000945e-08j,  2.88772402e+08-1.11353778e-08j,
        2.32705659e+08-7.73320880e-09j,  2.00352034e+08+5.77387821e-10j]),
 'bk_shot': array([1.89812860e+08-2.11770798e-09j, 2.06798899e+08-7.67983941e-10j,
       1.50707291e+08-4.82533270e-10j, 1.24950413e+08-4.09279101e-10j,
       1.06980043e+08-2.03918312e-10j, 9.44867847e+07-3.82516272e-10j,
       8.04644910e+07+1.48562912e-10j, 6.84708346e+07-5.49508290e-11j,
       5.94749097e+07+3.25234856e-11j, 5.12239797e+07+2.63271010e-11j]),
 'k1_bin': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
 'k1_eff': array([0.01149964, 0.02047114, 0.03052422, 0.04062536, 0.05033379,
       0.06040971, 0.07034279, 0.08025683, 0.09014322, 0.10008191]),
 'k2_bin': array([0.01, 0.

The saved file contains a header with summary information about the parameters
used and the input data as well as intermediary quantities:

In [10]:
# DEMO
with open("bk000_diag.txt", 'r') as results_file:
    print(results_file.read())

# Catalogue source: extfile:mock_catalogue_sim.dat
# Catalogue size: ntotal = 499214, wtotal = 499214.000, wstotal = 499214.000
# Catalogue particle extents: ([0.002, 999.998], [0.001, 999.999], [0.000, 1000.000])
# Box size: [1000.000, 1000.000, 1000.000]
# Box alignment: centre
# Mesh number: [64, 64, 64]
# Mesh assignment and interlacing: cic, False
# Normalisation factor: 8.037846928e+00 (particle)
# Normalisation factor alternatives: 8.037846928e+00 (particle), 2.208590772e+00 (mesh), 0.000000000e+00 (mesh-mixed; n/a)
# [0] k1_cen, [1] k1_eff, [2] nmodes_1, [3] k2_cen, [4] k2_eff, [5] nmodes_2, [6] Re{bk000_raw}, [7] Im{bk000_raw}, [8] Re{bk000_shot}, [9] Im{bk000_shot}
1.000000000e-02	1.149964290e-02	        56	1.000000000e-02	1.149964290e-02	        56	-9.766343488e+08	 2.009053768e-07	 1.898128595e+08	-2.117707984e-09
2.000000000e-02	2.047114034e-02	       194	2.000000000e-02	2.047114034e-02	       194	 5.623084213e+09	-4.851025251e-07	 2.067988992e+08	-7.679839408e-10
3.000000

```{seealso}
[Clustering Measurements](./Measurements.ipynb) for more details.
```

In [11]:
# Hide cell
!rm params_template.yml bk000_diag.txt

## Running the C++ program

For most users, the C++ code serves as the backend for the Python package
and the only public routine is the compiled binary executable program
``triumvirate`` which should be run with an external parameter file.

### Parameter configuration

A template parameter file [``"params_template.ini"``](../_static/params_template.ini)
can be downloaded, but the relevant parameters need to be changed accordingly
for your use.

```{seealso}
[Parameter Configuration](./Configuration.ipynb) for more details.
```

In [12]:
import textwrap

params_template = textwrap.dedent(
    """\
    # -- I/O -----------------------------------------------------------------

    # Directories for input catalogue(s) and output measurement(s).
    # The paths can be either absolute or relative to the working directory.
    # If unset, the current working directory is assumed.
    catalogue_dir =
    measurement_dir =

    # Filenames (with extension) of input catalogues.  These are relative
    # to the catalogue directory.
    data_catalogue_file = mock_catalogue_sim.dat
    rand_catalogue_file =

    # Field names of catalogue data columns as a comma-separated list without
    # space in the order of appearance.  Only data columns with the following
    # field names are read from the input catalogue(s),
    #   {'x', 'y', 'z', 'nz', 'ws', 'wc'},
    # where 'x', 'y', 'z' are the Cartesian coordinates, 'nz' is the
    # redshift-dependent number density, 'ws' is the total sample weight
    # (including e.g. completeness weights), and 'wc' is the total clustering
    # weight (including e.g. optimality weights).
    catalogue_columns = x,y,z

    # Tags to be appended as an input/output filename suffix.
    output_tag =


    # -- Mesh sampling -------------------------------------------------------

    # Box size in each dimension (in length units).
    boxsize_x = 1000.
    boxsize_y = 1000.
    boxsize_z = 1000.

    # Grid cell number in each dimension.
    ngrid_x = 64
    ngrid_y = 64
    ngrid_z = 64

    # Box expansion factor: {1. (default), >1.}.
    # The box is expanded by this factor in each dimension when
    # box size is not set.
    expand: 1.

    # Box alignment: {'centre' (default), 'pad'}.
    # The catalogues are either centred or padded from the
    # mininum-coordinate corner.
    alignment = centre

    # Padding scale: {'box' (default), 'grid'}.
    # The padding scale is either the box size or the grid cell size.
    # Only applicable if `alignment` is set to 'pad'.
    padscale = box

    # Padding factor (as a multiple of the size of padding scale).
    # Only applicable if `alignment` is set to 'pad'.
    padfactor =

    # Mesh assignment scheme: {'ngp', 'cic', 'tsc' (default), 'pcs'}.
    assignment = tsc

    # Interlacing switch: {'true'/'on', 'false'/'off' (default)}.
    # The switch is overridden to 'false' when measuring three-point statistics.
    interlace = false


    # -- Measurements --------------------------------------------------------

    # Type of catalogue(s): {'survey', 'random', 'sim', 'none'}.
    # If of type 'sim', global-plane-parallel measurements are made;
    # otherwise, local-plane-parallel measurements are made. [mandatory]
    catalogue_type = sim

    # Type of measurement: {
    #   'powspec', '2pcf', '2pcf-win',
    #   'bispec', '3pcf', '3pcf-win', '3pcf-win-wa',
    #   'modes', 'pairss'
    # }. [mandatory]
    statistic_type = bispec

    # Degrees of the multipoles.
    ell1 = 0
    ell2 = 0
    ELL = 0

    # Orders of wide-angle corrections.
    i_wa =
    j_wa =

    # Form of three-point statistic measurements:
    # {'full', 'diag' (default), 'off-diag', 'row'}.
    form = diag

    # Normalisation convention: {
    #   'none', 'particle' (default), 'mesh',
    #   'mesh-mixed' (two-point statistics only)
    # }.
    norm_convention = particle

    # Binning scheme: {'lin' (default), 'log', 'linpad', 'logpad', 'custom'}.
    binning = lin

    # Minimum and maximum of the range of measurement scales.
    # The binning coordinate is either wavenumbers in Fourier space,
    # or separations in configuration space. [mandatory]
    bin_min = 0.005
    bin_max = 0.105

    # Number of measurement bins (i.e. data vector dimension).
    # Must be >=2, or >=7 if padding is used. [mandatory]
    num_bins = 10

    # Fixed bin index when the `form` of three-point statistics measurements
    # is set to 'off-diag' or 'row'.  If 'off-diag', the bin index is the
    # positive off-diagonal index; if `form` is set to 'row', the bin index
    # is the row index.
    idx_bin = 0


    # -- Misc ----------------------------------------------------------------

    # FFTW scheme: {'estimate', 'measure' (default), 'patient'}.
    # This corresponds to the FFTW planner flags.
    fftw_scheme: measure

    # Use FFTW wisdom: {false (default)/off, <path-to-dir>}.
    # If not `false` or non-empty, then this is the path to
    # the FFTW wisdom directory; the FFTW wisdom file is either imported
    # from there or exported there if the wisdom file does not yet exist;
    # `fftw_scheme` must be set to 'measure' or higher (i.e. 'patient').
    use_fftw_wisdom: false

    # Save binning details to file:
    # {'true', 'false' (default), <relpath-to-file>}.
    # If a path is provided, it is relative to the measurement directory.
    # An empty path is equivalent to 'false'.
    save_binned_vectors = false

    # Logging verbosity level: a non-negative integer.
    # Typical values are: {
    #   0 (NSET, unset), 10 (DBUG, debug), 20 (STAT, status) (default),
    #   30 (INFO, info), 40 (WARN, warning), 50 (ERRO, error)
    # }.
    verbose = 20

    # Progress bar display switch: {true/on, false/off (default), <%-points>}.
    # If a float is provided, the progress bar is update/displayed
    # at that percentage-point interval, e.g. 10. for every 10%.
    progbar: off
    """
)

with open("params_template.ini", 'w', encoding='ascii') as params_template_file:
    params_template_file.write(params_template)

For the example below, we have modified the following lines in the template
parameter file:

```{code-block} ini
:caption: params_template.ini

data_catalogue_file = mock_catalogue_sim.dat

catalogue_columns = x,y,z

boxsize_x = 1000.
boxsize_y = 1000.
boxsize_z = 1000.

ngrid_x = 64
ngrid_y = 64
ngrid_z = 64

catalogue_type = sim

statistic_type = bispec

ell1 = 0
ell2 = 0
ELL = 0

bin_min = 0.005
bin_max = 0.105

num_bins = 10
```

### Executing the program

To run the 'black-box' C++ program, execute
```{code-block} console
$ <path-to-executable> <path-to-parameter-file>
```
in the shell.

Assuming the compiled binary executable is in the default build directory and
the parameter file is in the current working directory which is the repository
directory root, we execute the following to make the same measurement
as before:
```{code-block} console
$ build/bin/triumvirate params_template.ini
```

In [13]:
%%bash

mkdir -p build/bin/
cp ../../../build/bin/triumvirate build/bin/triumvirate
build/bin/triumvirate params_template.ini

PROGRAM LOG >

[2024-08-19 23:53:19 (+00:00:00) STAT] [MAIN:TRV:A] Parameters and source data are being initialised.
[2024-08-19 23:53:19 (+00:00:00) STAT] [MAIN:TRV:A] Reading parameters...
[2024-08-19 23:53:19 (+00:00:00) STAT] Parameters validated.
[2024-08-19 23:53:19 (+00:00:00) STAT] Parameters validated.
[2024-08-19 23:53:19 (+00:00:00) INFO] Check used-parameter file for reference: .//parameters_used
[2024-08-19 23:53:19 (+00:00:00) STAT] [MAIN:TRV:A] ... read parameters.
[2024-08-19 23:53:19 (+00:00:00) STAT] [MAIN:TRV:A] Reading catalogues...
[2024-08-19 23:53:19 (+00:00:00) WARN] Catalogue 'nz' field is unavailable and will be set to the mean density in the bounding box (source=extfile:mock_catalogue_sim.dat).
[2024-08-19 23:53:20 (+00:00:01) INFO] Catalogue loaded: ntotal = 499214, wtotal = 499214.000, wstotal = 499214.000 (source=extfile:mock_catalogue_sim.dat).
[2024-08-19 23:53:20 (+00:00:01) INFO] Extents of particle coordinates: {'x': (-499.998, 499.998 | 999.996), 'y'

As before, the saved file contains a header with summary information about the
parameters used and the input data as well as intermediary quantities:

In [14]:
# DEMO
with open("bk000_diag", 'r') as results_file:
    print(results_file.read())

# Catalogue source: extfile:mock_catalogue_sim.dat
# Catalogue size: ntotal = 499214, wtotal = 499214.000, wstotal = 499214.000
# Catalogue particle extents: [(0.000, 1000.000), (0.000, 999.996), (0.001, 999.997)]
# Box size: [1000.000, 1000.000, 1000.000]
# Box alignment: centre
# Mesh number: [64, 64, 64]
# Mesh assignment and interlacing: tsc, false
# Normalisation factor: 8.037846928e+00 (particle)
# Normalisation factor alternatives: 8.037846928e+00 (particle), 3.089710869e+00 (mesh), 0.000000000e+00 (mesh-mixed)
# [0] k1_cen, [1] k1_eff, [2] nmodes_1, [3] k2_cen, [4] k2_eff, [5] nmodes_2, [6] Re{bk000_raw}, [7] Im{bk000_raw}, [8] Re{bk000_shot}, [9] Im{bk000_shot}
1.000000000e-02	1.149964290e-02	        56	1.000000000e-02	1.149964290e-02	        56	-9.702292027e+08	-9.532246041e-07	 1.898133758e+08	 3.469509921e-09
2.000000000e-02	2.047114034e-02	       194	2.000000000e-02	2.047114034e-02	       194	 5.617152234e+09	-2.008673590e-07	 2.067769125e+08	 2.585273166e-09
3.000000000e-

In addition, the interpreted/processed parameters used by the program are
saved to the ``"parameters_used*"`` file in the specified measurement
directory (the current working directory here), where the wildcard asterisk
is the output tag (empty here) specified in the input parameter file.

In [15]:
# DEMO
with open("parameters_used", 'r') as used_params_file:
    print(used_params_file.read())

catalogue_dir = 
measurement_dir = ./
data_catalogue_file = mock_catalogue_sim.dat
rand_catalogue_file = 
catalogue_columns = x,y,z
output_tag = 
boxsize_x = 1000.000
boxsize_y = 1000.000
boxsize_z = 1000.000
ngrid_x = 64
ngrid_y = 64
ngrid_z = 64
volume = 1.000000e+09
nmesh = 262144
expand = 1.0000
alignment = centre
padscale = box
padfactor = 0.0000
assignment = tsc
interlace = false
assignment_order = 3
catalogue_type = sim
statistic_type = bispec
npoint = 3pt
space = fourier
ell1 = 0
ell2 = 0
ELL = 0
i_wa = 0
j_wa = 0
form = diag
norm_convention = particle
binning = lin
shape = diag
bin_min = 0.0050
bin_max = 0.1050
num_bins = 10
idx_bin = 0
fftw_scheme = measure
use_fftw_wisdom = 
fftw_wisdom_file_f = 
fftw_wisdom_file_b = 
save_binned_vectors = 
progbar = true
verbose = 20
fftw_planner_flag = 0



In [16]:
# Hide cell.
!rm -r ./build/ ./params_template.ini ./bk000_diag ./parameters_used*